In [1]:
import pandas as pd
import numpy  as np

## Preparing metadata for measurements

In [2]:
# prepare isd-history metadata
metadata = pd.read_csv("../data/isd-history.csv")
metadata.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29775 entries, 0 to 29774
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   USAF          29775 non-null  object 
 1   WBAN          29775 non-null  int64  
 2   STATION NAME  28986 non-null  object 
 3   CTRY          28838 non-null  object 
 4   STATE         6743 non-null   object 
 5   ICAO          10891 non-null  object 
 6   LAT           28571 non-null  float64
 7   LON           28570 non-null  float64
 8   ELEV(M)       28477 non-null  float64
 9   BEGIN         29775 non-null  int64  
 10  END           29775 non-null  int64  
dtypes: float64(3), int64(3), object(5)
memory usage: 2.5+ MB


In [3]:
# let's look first few examples 
metadata.head()

,USAF,WBAN,STATION NAME,CTRY,STATE,ICAO,LAT,LON,ELEV(M),BEGIN,END
0,007018,99999,WXPOD 7018,NaN,NaN,NaN,0.00,0.000,7018.0,20110309,20130730
1,007026,99999,WXPOD 7026,AF,NaN,NaN,0.00,0.000,7026.0,20120713,20170822
2,007070,99999,WXPOD 7070,AF,NaN,NaN,0.00,0.000,7070.0,20140923,20150926
3,008260,99999,WXPOD8270,NaN,NaN,NaN,0.00,0.000,0.0,19960101,20100731
4,008268,99999,WXPOD8278,AF,NaN,NaN,32.95,65.567,1156.7,20100519,20120323


In [4]:
# selecting required columns
metadata = metadata[["USAF", "WBAN", "CTRY", "STATION NAME"]]
metadata.rename(columns = {'USAF':'stn', 'WBAN': 'wban', 'CTRY': 'ctry', 'STATION NAME':'station_name'}, inplace=True) 

metadata.head()


,stn,wban,ctry,station_name
0,007018,99999,NaN,WXPOD 7018
1,007026,99999,AF,WXPOD 7026
2,007070,99999,AF,WXPOD 7070
3,008260,99999,NaN,WXPOD8270
4,008268,99999,AF,WXPOD8278


In [5]:
# to make sure we always have mapping country and station name
metadata.dropna(subset=["ctry", "station_name"], inplace=True)

metadata.head()

,stn,wban,ctry,station_name
1,007026,99999,AF,WXPOD 7026
2,007070,99999,AF,WXPOD 7070
4,008268,99999,AF,WXPOD8278
5,008307,99999,AF,WXPOD 8318
10,010000,99999,NO,BOGUS NORWAY


In [6]:
# replacing missing values with np.nan
metadata.replace([99999, "999999"], np.nan, inplace=True)

metadata.head()

,stn,wban,ctry,station_name
1,007026,NaN,AF,WXPOD 7026
2,007070,NaN,AF,WXPOD 7070
4,008268,NaN,AF,WXPOD8278
5,008307,NaN,AF,WXPOD 8318
10,010000,NaN,NO,BOGUS NORWAY


In [7]:
# we need wban or stn available always to link metadata to actual measurements
metadata = metadata[~(metadata.stn.isin([np.nan]) & metadata.wban.isin([np.nan]))]

metadata.head()

,stn,wban,ctry,station_name
1,007026,NaN,AF,WXPOD 7026
2,007070,NaN,AF,WXPOD 7070
4,008268,NaN,AF,WXPOD8278
5,008307,NaN,AF,WXPOD 8318
10,010000,NaN,NO,BOGUS NORWAY


In [8]:
# there is a quite amount of not unique stns and wbans thus we need to combine them to form one unique station id
metadata["stid"] = metadata.stn.apply(lambda x: str(x)) + metadata.wban.apply(lambda x: str(x)) 
metadata.drop(["stn", "wban"], axis=1, inplace=True)

metadata.head()

,ctry,station_name,stid
1,AF,WXPOD 7026,007026nan
2,AF,WXPOD 7070,007070nan
4,AF,WXPOD8278,008268nan
5,AF,WXPOD 8318,008307nan
10,NO,BOGUS NORWAY,010000nan


In [9]:
metadata.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 28781 entries, 1 to 29774
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   ctry          28781 non-null  object
 1   station_name  28781 non-null  object
 2   stid          28781 non-null  object
dtypes: object(3)
memory usage: 899.4+ KB


In [12]:
# saving results for further use
metadata.to_csv("../data/metadata.csv", index=False)